In [2]:
%run packages.py

/Users/mcbookairdebat/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
from data_import_enginnering import DataImportEnginnering
from metric_selection import Pair_Selection
from data_visualization import Data_Visualization
import importlib

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# 1. DATA

## 1.1. Importation of data into a dictionary

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tables = pd.read_html(url)
Stock_list = pd.DataFrame(tables[0])
Stock_list.drop(['Security','GICS Sub-Industry','Headquarters Location','CIK','Founded'],axis = 1, inplace=True)
Stock_list['Date added'] = pd.to_datetime(Stock_list['Date added'])
Stock_list = Stock_list[( Stock_list['Date added'] < '2023-12-31') & (Stock_list['Date added']>'2015-01-01')][['Symbol','GICS Sector']]

In [5]:
stock_list = list(Stock_list['Symbol'].unique())
Stock_list_sector = Stock_list.groupby('GICS Sector')['Symbol'].unique().to_dict()
sector_list = list(Stock_list_sector.keys())
pairs = {k:[tuple(sorted((i,j))) for i in Stock_list_sector[k] for j in Stock_list_sector[k] if j!=i] for k in sector_list}
pairs = {k :sorted(list(set(pairs[k]))) for k in sector_list}

In [31]:
start_date = '2018-01-01'
end_date = '2022-12-31'
period = '1d'
colone = ['Close']


In [32]:
Enginner = DataImportEnginnering(stock_list,start_date=start_date,end_date=end_date,period=period,colone=colone)

In [33]:
raw_data = Enginner.RawData()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

## 1.2. Normalisation of data

In [34]:
raw_data_normalized = {}
for key, value in raw_data.items():
    raw_data_normalized[key] = Enginner.normalize(value)

raw_data_normalized

{'AOS': Price          Close
 Ticker           AOS
 Date                
 2018-01-02  0.454586
 2018-01-03  0.461197
 2018-01-04  0.466378
 2018-01-05  0.480672
 2018-01-08  0.486032
 ...              ...
 2022-12-23  0.465824
 2022-12-27  0.480853
 2022-12-28  0.463287
 2022-12-29  0.480658
 2022-12-30  0.472460
 
 [1259 rows x 1 columns],
 'AMD': Price          Close
 Ticker           AMD
 Date                
 2018-01-02  0.009516
 2018-01-03  0.013256
 2018-01-04  0.016997
 2018-01-05  0.015422
 2018-01-08  0.018047
 ...              ...
 2022-12-23  0.360874
 2022-12-27  0.352671
 2022-12-28  0.348077
 2022-12-29  0.362843
 2022-12-30  0.362515
 
 [1259 rows x 1 columns],
 'ABNB': Price          Close
 Ticker          ABNB
 Date                
 2020-12-10  0.463119
 2020-12-11  0.422479
 2020-12-14  0.353629
 2020-12-15  0.314924
 2020-12-16  0.413100
 ...              ...
 2022-12-23  0.020543
 2022-12-27  0.007443
 2022-12-28  0.000000
 2022-12-29  0.020395
 2022-12-30  0.02240

In [35]:
dataframe_normalized = Enginner.Dictionnary_to_Dataframe(raw_data_normalized)
raw_dataframe = Enginner.Dictionnary_to_Dataframe(raw_data)


/Users/mcbookairdebat/Desktop/Projet/project/GitHub/Pair_trading_strategy/Pair_trading/data_import_enginnering.py:40: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/Users/mcbookairdebat/Desktop/Projet/project/GitHub/Pair_trading_strategy/Pair_trading/data_import_enginnering.py:40: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/Users/mcbookairdebat/Desktop/Projet/project/GitHub/Pair_trading_strategy/Pair_trading/data_import_enginnering.py:40: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of call

In [36]:
dataframe_normalized= dataframe_normalized.stack().stack().droplevel(level =2).unstack()
raw_dataframe= raw_dataframe.stack().stack().droplevel(level =2).unstack()

/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_75189/628406389.py:1: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.

/var/folders/lc/t3q2r39x05ndfqz7hm78wmkc0000gn/T/ipykernel_75189/628406389.py:2: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



## 1.3 Correlation Matrix

In [37]:
q = raw_dataframe.shape[1]/raw_dataframe.shape[0]
data = raw_dataframe.copy()
data = np.log(data/data.shift(1))
data = data.apply(lambda x: x-x.mean())
data = data.iloc[1:,:]


In [38]:
correlation_matrix = data.corr()

# 2.Reduction Dimension with PCA and Marcenko Pastur 

## Marcenko pastur to keep only the rights one

In [39]:

def mpPDF(var,q):
    e_min,e_max = var* (1 -(1/q)**.5),var* (1 + (1/q)**.5)
    return e_min,e_max

from sklearn.neighbors._kde import KernelDensity
def getPCA(matrix):
    eVal,eVec = np.linalg.eigh(matrix)
    indices = eVal.argsort()[::-1]
    eVal,eVec = eVal[indices],eVec[:,indices]
    eVal = np.diagflat(eVal)
    return eVal,eVec

def fitKDE(obs,bWidth =.25,kernel = 'gaussian', x=None):
    if len(obs.shape)==1: obs =obs.reshape(-1,1)
    kde = KernelDensity(bandwidth=bWidth,kernel=kernel).fit(obs)
    if x is None : x=np.unique(obs).reshape(-1,1)
    if len(x.shape)==1 : x=x.reshape(-1,1)
    logProb = kde.score_samples(x)
    pdf = pd.Series(np.exp(logProb), index = x.flatten())
    return pdf

x = np.random.normal(size=(1000,1000))
eVal0,eVec0 = getPCA(correlation_matrix)
e_min,e_max = mpPDF(1.,raw_dataframe.shape[1]/raw_dataframe.shape[0])
mask = (np.diag(eVal0)>e_max)

In [40]:
mt =mask*np.diag(eVal0)
treshold = (mt>0).sum()
eVec0 = pd.DataFrame(eVec0).T


In [41]:
pca_proj = eVec0.iloc[:,:3]
new_corr = np.dot(correlation_matrix.T,pca_proj)
new_corr = pd.DataFrame(new_corr,index = correlation_matrix.index,columns =['X','Y','Z'])

# 3.Applying Optic/ DBScan and construction pair 

In [42]:
# Research 
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler


X = StandardScaler().fit_transform(new_corr)
X =pd.DataFrame(X,index = new_corr.index, columns = ['X','Y','Z'])
X

,X,Y,Z
Ticker,,,
AOS,0.692568,0.181354,1.096736
AMD,-0.829880,1.575946,-1.069017
ALB,0.017890,0.607129,0.694049
ARE,-1.175834,0.356609,-0.990398
ALGN,1.001482,0.124163,-0.019021
...,...,...,...
CARR,0.412721,-1.199166,0.897343
OTIS,-0.485802,-1.087351,0.624783
ABNB,-0.643000,-1.074405,-0.325686


In [43]:
import numpy as np

from sklearn import metrics
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.3, min_samples=3).fit(X)
labels = db.labels_
X['label'] =labels

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Estimated number of clusters: 14
Estimated number of noise points: 102


In [44]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter_3d(X, x='X', y='Y', z='Y',color='label')
fig.show()

In [45]:
X.reset_index(inplace=True, drop = False)


In [46]:
f_dic ={}
for tick in X.label.unique():
    if tick ==-1:
        pass
    else:
        f_dic[tick] = X[X['label']==tick]['Ticker']

In [47]:
pairs=[]
for val in f_dic.values():
    val1 = val.copy()
    for stock1 in val1:
        pairs.extend([(stock1,stock2)  for stock2 in val1 if stock2!=stock1])
        val1 = val1[1:]

# 4.Rules for pair selection 

## Check for cointegration

In [48]:
# Engle granger 
# first cointegration step 
Metric = Pair_Selection(dataframe_normalized,number_of_pair=2,stock_list_sector=Stock_list_sector,sector_list=sector_list,sector_neutral = True)

In [49]:
selected_pair = Metric.paire_selection_2(pairs,p=0.05)

## Check that the spread is mean reverting

In [57]:
from Spread import Spread
s = Spread(raw_data_normalized)
spreads = s.dollar_neutral_spread()

KeyError: 'A'

In [51]:
# Hurst Exponent 
#Implementation of the R/S method 
lenghts = np.arange(5,100,5)

## Check that the spread is tradable 

In [52]:
#

## Check spread reversion is consistent 


In [53]:
# mean_cross superior to 12 